In [2]:
# Imports

import os
import pandas as pd
import json

In [3]:
# Constants

DATASET_PATH = os.path.join("data", "datageneration", "datasets", "dataset.parquet")
DIFFS_PATH = os.path.join("data", "datageneration", "diffs.json")

In [9]:
# Show the dataset

df = pd.read_parquet(DATASET_PATH)

df.head(5)

,author,date,timezone,hash,message,mods,language,license,repo,original_message
0,652323,02.12.2017 23:37:47,-10800,eea4c9c55a01df00e5518ee497cc086ac5e163e0,fix ItemController template,"[{'change_type': 'MODIFY', 'old_path': 'templa...",Kotlin,Apache License 2.0,surfstudio/surfandroidstandard,fix ItemController template
1,652304,09.01.2018 21:25:28,-10800,1308f18b14675b25ce7ba79da37b397a1c55ef57,artifactory deploy,"[{'change_type': 'MODIFY', 'old_path': 'build....",Kotlin,Apache License 2.0,surfstudio/surfandroidstandard,artifactory deploy
2,652323,26.01.2018 10:03:56,0,e4b446cd3dd2710f37073a3feba97acf7e790924,bitbucket-pipelines.yml deleted online with Bi...,"[{'change_type': 'DELETE', 'old_path': 'bitbuc...",Kotlin,Apache License 2.0,surfstudio/surfandroidstandard,bitbucket-pipelines.yml deleted online with Bi...
3,652304,30.01.2018 17:46:06,-10800,f0ceee6b95fcad7e119d90277b27d5d972ce23c7,fix. access level,"[{'change_type': 'MODIFY', 'old_path': 'filest...",Kotlin,Apache License 2.0,surfstudio/surfandroidstandard,fix. access level
4,652303,13.03.2018 11:59:28,-10800,de2e1d1225e2b18c9998940641968128ed9a77ae,#ANDDEP-97 made every instance of ItemControll...,"[{'change_type': 'MODIFY', 'old_path': 'easyad...",Kotlin,Apache License 2.0,surfstudio/surfandroidstandard,#ANDDEP-97 made every instance of ItemControll...


In [5]:
# Show the diffs

mods = df["mods"]

mods[0]

array([{'change_type': 'MODIFY', 'old_path': 'templates/file/surf/ItemController/globals.xml.ftl', 'new_path': 'templates/file/surf/ItemController/globals.xml.ftl', 'diff': '<global id="nameParam" value="${extractLetters(nameTypeData?lower_case)}" />\n<global id="defPostfixController" value="ItemController" />\n- <global id="ktOrJavaExt" type="string" value="${generateKotlin?string(\'kt\',\'java\')}" />\n</globals>\n'},
       {'change_type': 'MODIFY', 'old_path': 'templates/file/surf/ItemController/recipe.xml.ftl', 'new_path': 'templates/file/surf/ItemController/recipe.xml.ftl', 'diff': '<#import "root://activities/common/kotlin_macros.ftl" as kt>\n<recipe>\n<@kt.addAllKotlinDependencies />\n- <instantiate from="src/app_package/Controller.${ktOrJavaExt}.ftl"\n- to="${escapeXmlAttribute(srcOut)}/${nameController}${defPostfixController}.${ktOrJavaExt}" />\n- <open file="${escapeXmlAttribute(srcOut)}/${nameController}${defPostfixController}.${ktOrJavaExt}" />\n+\n+ <#if generateKotlin>\n

In [6]:
# Filter the mods for only Kotlin files

filtered_mods = df['mods'].apply(
    lambda mod_list: [
        mod for mod in mod_list if str(mod['old_path']).endswith('.kt') or str(mod['new_path']).endswith('.kt')
    ]
)

filtered_mods = filtered_mods[filtered_mods.apply(lambda x: len(x) > 0)]
filtered_mods.iloc[0]

[{'change_type': 'MODIFY',
  'old_path': 'template/app-injector/src/main/java/ru/surfstudio/standard/app_injector/AppComponent.kt',
  'new_path': 'template/app-injector/src/main/java/ru/surfstudio/standard/app_injector/AppComponent.kt',
  'diff': '@@ -8,6 +8,7 @@ import ru.surfstudio.android.core.app.ActiveActivityHolder\nimport ru.surfstudio.android.core.app.StringsProvider\nimport ru.surfstudio.android.core.ui.navigation.activity.navigator.GlobalNavigator\nimport ru.surfstudio.android.dagger.scope.PerApplication\n+import ru.surfstudio.android.notification.PushHandler\nimport ru.surfstudio.android.notification.interactor.push.storage.FcmStorage\nimport ru.surfstudio.android.rx.extension.scheduler.SchedulersProvider\nimport ru.surfstudio.android.shared.pref.NO_BACKUP_SHARED_PREF\n@@ -20,6 +21,7 @@ import ru.surfstudio.standard.app_injector.network.OkHttpModule\nimport ru.surfstudio.standard.app_injector.network.cache.CacheModule\nimport ru.surfstudio.standard.app_injector.ui.notificati

In [7]:
# Convert filtered mods to list of raw diffs

diffs = filtered_mods.apply(
    lambda mod_list: '\n\n'.join([
        f"""diff --git a/{mod['old_path']} b/{mod['new_path']}
index 0000000..0000000 100644
--- a/{mod['old_path']}
+++ b/{mod['new_path']}
{mod['diff']}
"""
        for mod in mod_list
    ])
).tolist()

diffs[0]

'diff --git a/template/app-injector/src/main/java/ru/surfstudio/standard/app_injector/AppComponent.kt b/template/app-injector/src/main/java/ru/surfstudio/standard/app_injector/AppComponent.kt\nindex 0000000..0000000 100644\n--- a/template/app-injector/src/main/java/ru/surfstudio/standard/app_injector/AppComponent.kt\n+++ b/template/app-injector/src/main/java/ru/surfstudio/standard/app_injector/AppComponent.kt\n@@ -8,6 +8,7 @@ import ru.surfstudio.android.core.app.ActiveActivityHolder\nimport ru.surfstudio.android.core.app.StringsProvider\nimport ru.surfstudio.android.core.ui.navigation.activity.navigator.GlobalNavigator\nimport ru.surfstudio.android.dagger.scope.PerApplication\n+import ru.surfstudio.android.notification.PushHandler\nimport ru.surfstudio.android.notification.interactor.push.storage.FcmStorage\nimport ru.surfstudio.android.rx.extension.scheduler.SchedulersProvider\nimport ru.surfstudio.android.shared.pref.NO_BACKUP_SHARED_PREF\n@@ -20,6 +21,7 @@ import ru.surfstudio.stan

In [8]:
# Save the first 100 diffs to json file
json_str = json.dumps(diffs[:100])

dir_path = os.path.dirname(DIFFS_PATH)
os.makedirs(dir_path, exist_ok=True)

with open(DIFFS_PATH, 'w') as f:
    f.write(json_str)